## MNIST Clan-ssifier Model 2 - CNN Model


In [ ]:
# !pip install ezkl

In [1]:

# install ezkl

import ezkl
import os
import json
import time
import random
import logging


# uncomment for more descriptive logging
FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.INFO)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # Convolutional encoder
        self.conv1 = nn.Conv2d(1, 6, 5)  # 1 input channel, 6 output channels, 5x5 kernel
        self.conv2 = nn.Conv2d(6, 16, 5) # 6 input channels, 16 output channels, 5x5 kernel

        # Fully connected layers / Dense block
        self.fc1 = nn.Linear(16 * 4 * 4, 120) 
        self.fc2 = nn.Linear(120, 84)         # 120 inputs, 84 outputs
        self.fc3 = nn.Linear(84, 10)          # 84 inputs, 10 outputs (number of classes)

    def forward(self, x):
        # Convolutional block
        x = F.avg_pool2d(F.sigmoid(self.conv1(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool
        x = F.avg_pool2d(F.sigmoid(self.conv2(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool

        # Flattening
        x = x.view(x.size(0), -1)

        # Fully connected layers
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = self.fc3(x)  # No activation function here, will use CrossEntropyLoss later
        return x


In [4]:
import numpy as np
import os
import torch
from torchvision.datasets import mnist
from torch.nn import CrossEntropyLoss
from torch.optim import Adam  # Import Adam
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

def normalize_img(image, label):
  return torch.round(image), label

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
batch_size = 256
train_dataset = mnist.MNIST(root='./train', train=True, transform=ToTensor(), download=True)
test_dataset = mnist.MNIST(root='./test', train=False, transform=ToTensor(), download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)
model = LeNet().to(device)
adam = Adam(model.parameters())  # Using Adam with a learning rate of 1e-3
loss_fn = CrossEntropyLoss()
all_epoch = 25
prev_acc = 0

cpu


In [9]:
from torchsummary import summary


In [10]:
summary(model,(1,28,28))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 24, 24]             156
            Conv2d-2             [-1, 16, 8, 8]           2,416
            Linear-3                  [-1, 120]          30,840
            Linear-4                   [-1, 84]          10,164
            Linear-5                   [-1, 10]             850
Total params: 44,426
Trainable params: 44,426
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.04
Params size (MB): 0.17
Estimated Total Size (MB): 0.21
----------------------------------------------------------------


In [3]:

for current_epoch in range(all_epoch):
    model.train()
    for idx, (train_x, train_label) in enumerate(train_loader):
        train_x = train_x.to(device)
        # normalize the image to 0 or 1 to reflect the inputs from the drawing board
        train_x = train_x.round()
        train_label = train_label.to(device)
        adam.zero_grad()  # Use adam optimizer
        predict_y = model(train_x.float())
        loss = loss_fn(predict_y, train_label.long())
        loss.backward()
        adam.step()  # Use adam optimizer
    all_correct_num = 0
    all_sample_num = 0
    model.eval()

    for idx, (test_x, test_label) in enumerate(test_loader):
        test_x = test_x.to(device)
         # normalize the image to 0 or 1 to reflect the inputs from the drawing board
        test_x = test_x.round()
        test_label = test_label.to(device)
        predict_y = model(test_x.float()).detach()
        predict_y = torch.argmax(predict_y, dim=-1)
        current_correct_num = predict_y == test_label
        all_correct_num += np.sum(current_correct_num.to('cpu').numpy(), axis=-1)
        all_sample_num += current_correct_num.shape[0]
    acc = all_correct_num / all_sample_num
    print('test accuracy: {:.3f}'.format(acc), flush=True)
    if not os.path.isdir("models"):
        os.mkdir("models")
    torch.save(model, 'models/mnist_{:.3f}.pkl'.format(acc))
    prev_acc = acc


test accuracy: 0.723
test accuracy: 0.852


KeyboardInterrupt: 

In [35]:
# load the saved model checkpoint with highest accuracy
model = torch.load("models/mnist_0.979.pkl")
model.eval()

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [ ]:
import pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_recall_curve

# generate a confusion matrix
confusion_mtx = confusion_matrix(y_true, y_pred_classes)

# visualize confusion matrix with matplotlib and seaborn
fig, ax = plt.subplots(figsize=(15, 10))
ax = sns.heatmap(confusion_mtx, annot=True, fmt="d", ax=ax, cmap="viridis")
ax.set_xlabel("Predicted Label")
ax.set_ylabel("True Label")
ax.set_title("Confusion Matrix");

In [42]:
# test the trained model

for idx, (train_x, train_label) in enumerate(train_loader):
    print(train_label) #, train_x)
    train_x = train_x.to(device)
    # normalize the image to 0 or 1 to reflect the inputs from the drawing board
    train_x = train_x.round()
    train_label = train_label.to(device)
    prediction = model(train_x.float())
    
    # print("prediction: ",prediction)
    predict_decoded = torch.argmax(prediction, dim=-1)
    print(predict_decoded)
    break

tensor([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0, 9, 1,
        1, 2, 4, 3, 2, 7, 3, 8, 6, 9, 0, 5, 6, 0, 7, 6, 1, 8, 7, 9, 3, 9, 8, 5,
        9, 3, 3, 0, 7, 4, 9, 8, 0, 9, 4, 1, 4, 4, 6, 0, 4, 5, 6, 1, 0, 0, 1, 7,
        1, 6, 3, 0, 2, 1, 1, 7, 9, 0, 2, 6, 7, 8, 3, 9, 0, 4, 6, 7, 4, 6, 8, 0,
        7, 8, 3, 1, 5, 7, 1, 7, 1, 1, 6, 3, 0, 2, 9, 3, 1, 1, 0, 4, 9, 2, 0, 0,
        2, 0, 2, 7, 1, 8, 6, 4, 1, 6, 3, 4, 5, 9, 1, 3, 3, 8, 5, 4, 7, 7, 4, 2,
        8, 5, 8, 6, 7, 3, 4, 6, 1, 9, 9, 6, 0, 3, 7, 2, 8, 2, 9, 4, 4, 6, 4, 9,
        7, 0, 9, 2, 9, 5, 1, 5, 9, 1, 2, 3, 2, 3, 5, 9, 1, 7, 6, 2, 8, 2, 2, 5,
        0, 7, 4, 9, 7, 8, 3, 2, 1, 1, 8, 3, 6, 1, 0, 3, 1, 0, 0, 1, 7, 2, 7, 3,
        0, 4, 6, 5, 2, 6, 4, 7, 1, 8, 9, 9, 3, 0, 7, 1, 0, 2, 0, 3, 5, 4, 6, 5,
        8, 6, 3, 7, 5, 8, 0, 9, 1, 0, 3, 1, 2, 2, 3, 3])
tensor([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0, 9, 1,
        1, 2, 4, 3, 2, 7, 3, 8, 6, 9, 0, 5, 6, 0, 7, 6, 1, 8, 7

In [43]:
# checking output logits of the model
prediction = model(train_x.float())
print(prediction)

tensor([[ -5.8988,  -2.4977,  -7.1422,  ...,  -2.6770,  -2.5237,  -1.8794],
        [  9.6284, -11.8113,  -1.4953,  ...,  -4.2926,  -2.7603,  -0.1944],
        [ -6.5762,  -3.2895,  -2.1786,  ...,  -1.0354,  -6.0388,  -0.8894],
        ...,
        [ -4.3042,  -2.8652,   7.9514,  ...,  -1.4231,  -2.2178,  -5.2027],
        [ -4.5936,  -3.0156,  -2.0817,  ...,  -2.7231,  -1.0167,  -1.1353],
        [ -5.3895,  -2.0177,  -1.9035,  ...,  -0.8790,   0.5988,  -1.9552]],
       grad_fn=<AddmmBackward0>)


# Implement Zero-Knowledge

In [44]:
# define files and respective paths

model_path = os.path.join('network_lenet.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('key.pk')
vk_path = os.path.join('key.vk')
settings_path = os.path.join('settings.json')
witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

In [47]:
# # Fetch a single data point from the train_dataset
# # Ensure train_dataset is already loaded and accessible
train_data_point, actual_label = next(iter(train_dataset))
train_data_point = train_data_point.unsqueeze(0)  # Add a batch dimension
print(train_data_point,actual_label)

tensor([[[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 

In [48]:
import torch
import json

model.eval()  # Set the model to evaluation mode


# Verify the device (CPU or CUDA) and transfer the data point to the same device as the model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
train_data_point = train_data_point.to(device)

# # Export the model to ONNX format
torch.onnx.export(model, train_data_point, model_path, export_params=True, opset_version=10, do_constant_folding=True, input_names=['input_0'], output_names=['output'])

# Convert the tensor to numpy array and reshape it for JSON serialization
x = train_data_point.cpu().detach().numpy().reshape([-1]).tolist()
data = {'input_data': [x]}
with open('input.json', 'w') as f:
    json.dump(data, f)

print(f"Model exported to {model_path} and input data saved to input.json")

Model exported to network_lenet.onnx and input data saved to input.json


In [49]:
train_data_point

tensor([[[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 

In [51]:
# check if training is false
model.training

False

# Test onnx model performance

In [52]:

def reverse_array_shape(x):
    original_shape = (1, 28, 28)  # Replace with the actual shape of the original tensor
    
    # Convert the list back to a numpy array
    numpy_array = np.array(x)

    # Reshape the numpy array to the original shape
    reshaped_array = numpy_array.reshape(original_shape)

    # Convert the numpy array back to a tensor
    train_data_point_reconstructed = torch.tensor(reshaped_array)
    return train_data_point_reconstructed

In [53]:
import ezkl

# define visibility or the model inputs and outputs
run_args = ezkl.PyRunArgs()
run_args.input_visibility = "private"
run_args.param_visibility = "fixed"
run_args.output_visibility = "public"
run_args.num_inner_cols = 2
run_args.variables = [("batch_size", 1)]

# Capture set of data points 
# Analyze caloibrated model with different data points eg- 1,3,30
num_data_points = 1

# Fetch 30 data points from the train_dataset
data_points = []
for i, (data_point, _) in enumerate(train_dataset):
    if i >= num_data_points:
        break
    data_points.append(data_point)

# Stack the data points to create a batch
train_data_batch = torch.stack(data_points)

# Add a batch dimension if not already present
if train_data_batch.dim() == 3:
    train_data_batch = train_data_batch.unsqueeze(0)

x = train_data_batch.cpu().detach().numpy().reshape([-1]).tolist()

data = dict(input_data = [x])

cal_path = os.path.join('cal_data.json')

# Serialize data into file:
json.dump( data, open(cal_path, 'w' ))

!RUST_LOG=trace
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path, py_run_args=run_args)
assert res == True

res = await ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources", scales=[2,7])
assert res == True

INFO ezkl.execute 2024-06-16 10:42:06,171 execute.rs:968 num calibration batches: 1
WARNING ezkl.circuit.table 2024-06-16 10:42:16,154 table.rs:165 Using 2 columns for non-linearity table.
WARNING ezkl.circuit.table 2024-06-16 10:42:16,156 table.rs:165 Using 2 columns for non-linearity table.
WARNING ezkl.circuit.table 2024-06-16 10:42:16,158 table.rs:165 Using 2 columns for non-linearity table.
WARNING ezkl.circuit.table 2024-06-16 10:42:16,160 table.rs:165 Using 2 columns for non-linearity table.
WARNING ezkl.circuit.table 2024-06-16 10:42:16,163 table.rs:165 Using 3 columns for non-linearity table.
WARNING ezkl.circuit.table 2024-06-16 10:42:16,168 table.rs:165 Using 3 columns for non-linearity table.
WARNING ezkl.circuit.table 2024-06-16 10:42:16,172 table.rs:165 Using 3 columns for non-linearity table.
WARNING ezkl.circuit.table 2024-06-16 10:42:16,176 table.rs:165 Using 3 columns for non-linearity table.
WARNING ezkl.circuit.table 2024-06-16 10:42:16,178 table.rs:165 Using 3 colu

In [55]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True
if res:
    print( "Circuit compiled successfully...!")

Circuit compiled successfully...!


In [56]:
# (Structured Reference String)srs path
srs_path="kzg.srs"

# get the SRS string
try:
  res = ezkl.get_srs(srs_path=srs_path,
                     settings_path=settings_path)
  if res:
    print("SRS was successfully fetched")
except Exception as e:
  print(f"An error occurred: {e}")

SRS was successfully fetched


INFO ezkl.execute 2024-06-16 10:43:34,728 execute.rs:672 SRS already exists at that path


INFO ezkl.execute 2024-06-16 10:43:35,558 execute.rs:570 read 16777476 bytes from file (vector of len = 16777476)
INFO ezkl.execute 2024-06-16 10:43:35,720 execute.rs:577 file hash: 41509f380362a8d14401c5ae92073154922fe23e45459ce6f696f58607655db7


In [57]:
# now generate the witness file
witness_path = "witness.json"

res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

# Test ZKML Model Predictions

In [58]:
# convert the quantized ezkl output to float value
with open(witness_path) as f:
    wit_data = json.load(f)
outputs = wit_data["outputs"]
with open(settings_path) as f:
    settings = json.load(f)
ezkl_outputs = [ezkl.felt_to_float(outputs[0][i], settings["model_output_scales"][0]) for i in range(10) ]
print( ezkl_outputs)


[-6.41455078125, -1.057373046875, -6.493408203125, 4.5693359375, -10.150146484375, 7.39404296875, -7.16015625, -1.08544921875, -3.865966796875, -2.697021484375]


In [59]:
# Convert the list to a tensor
ezkl_witnessed_prediction = torch.tensor(ezkl_outputs)

# Reshape the tensor if needed (not necessary in this case as it's already the correct shape)
# ezkl_outputs = ezkl_outputs.view(1, -1)  # Uncomment this line if ezkl_outputs should have shape (1, 10)

# Apply the model's prediction decoding (e.g., argmax to get the predicted class)
predict_decoded = torch.argmax(ezkl_witnessed_prediction, dim=-1)

# Print the predicted class
print(predict_decoded)

tensor(5)


In [60]:
# mock proof for sanity check
# if this is successfull we are good to setup the model
try:
  res = ezkl.mock(witness=witness_path,
                  model=compiled_model_path)
  if res:
    print("Mock proof run was successful")
except Exception as e:
  print(f"An error occurred: {e}")

INFO ezkl.execute 2024-06-16 10:46:50,466 execute.rs:1334 Mock proof


INFO ezkl.graph.model 2024-06-16 10:46:50,963 model.rs:1097 model layout...


Mock proof run was successful


In [61]:

# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING NEED FOR IMPLEMENTING ZK WITH ML

res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

INFO ezkl.pfsys.srs 2024-06-16 10:47:43,146 srs.rs:34 loading srs from "/home/gk/.ezkl/srs/kzg17.srs"
INFO ezkl.execute 2024-06-16 10:47:43,452 execute.rs:2482 downsizing params to 17 logrows
INFO ezkl.graph.model 2024-06-16 10:47:43,586 model.rs:1097 model layout...
INFO ezkl.pfsys 2024-06-16 10:48:06,332 mod.rs:512 VK took 22.878
INFO ezkl.graph.model 2024-06-16 10:48:06,345 model.rs:1097 model layout...
INFO ezkl.pfsys 2024-06-16 10:48:28,591 mod.rs:518 PK took 22.258
INFO ezkl.pfsys 2024-06-16 10:48:28,594 mod.rs:804 saving verification key 💾
INFO ezkl.pfsys 2024-06-16 10:48:28,603 mod.rs:809 done saving verification key ✅
INFO ezkl.pfsys 2024-06-16 10:48:28,604 mod.rs:787 saving proving key 💾
INFO ezkl.pfsys 2024-06-16 10:48:41,205 mod.rs:792 done saving proving key ✅


In [62]:
# GENERATE A PROOF

# proof path
proof_path = os.path.join('test.pf')

# generate proof
try:
    res = ezkl.prove(
        witness= witness_path,
        model =  compiled_model_path,
            pk_path=pk_path,
            proof_path= proof_path,
            srs_path="kzg.srs",
            proof_type="single")
    if res:
        print("Proof was successfully generated")
except Exception as e:
  print(f"An error occurred: {e}")

# check oif proof file is generated
print(res)
assert os.path.isfile(proof_path)

INFO ezkl.pfsys 2024-06-16 10:49:30,621 mod.rs:766 loading proving key from "key.pk"
INFO ezkl.pfsys 2024-06-16 10:49:33,752 mod.rs:775 done loading proving key ✅
INFO ezkl.pfsys.srs 2024-06-16 10:49:33,754 srs.rs:34 loading srs from "kzg.srs"
INFO ezkl.execute 2024-06-16 10:49:33,795 execute.rs:2482 downsizing params to 17 logrows
INFO ezkl.pfsys 2024-06-16 10:49:33,798 mod.rs:573 proof started...
INFO ezkl.graph.model 2024-06-16 10:49:33,890 model.rs:1097 model layout...
INFO ezkl.pfsys 2024-06-16 10:50:31,016 mod.rs:612 proof took 57.216


Proof was successfully generated
{'instances': [['5f99ffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430', '16efffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430', '1c98ffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430', '1c49000000000000000000000000000000000000000000000000000000000000', '9a5dffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430', '4e76000000000000000000000000000000000000000000000000000000000000', '718dffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430', 'a3eeffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430', '26c2ffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430', 'dad4ffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430']], 'proof': '0x0f0daf997aa2a7336b9bfed20848553e0e67a9a93db6ff1e43c4a226583768bf11a3dfc90d63b99ea86752ed86c032fce4bbb43896c5de8f84ad792bbe20aeaf1dd918b3d491ea074d6e3ab971da46592e438887c05f9ad2e4e35645027134bd0d812aac7acc2f8a29e0237d113ed9d9060baa8b4512aa955119b771e73342d50

In [63]:
# verify proof
try:
  res = ezkl.verify(proof_path=proof_path,
                    settings_path=settings_path,
                    vk_path=vk_path,
                    srs_path="kzg.srs")
  if res:
    print("Proof was successfully verified")
except Exception as e:
  print(f"An error occurred: {e}")

INFO ezkl.pfsys.srs 2024-06-16 10:58:11,548 srs.rs:23 loading srs from "kzg.srs"
INFO ezkl.execute 2024-06-16 10:58:11,629 execute.rs:2467 downsizing params to 17 logrows
INFO ezkl.pfsys 2024-06-16 10:58:11,632 mod.rs:743 loading verification key from "key.vk"
INFO ezkl.pfsys 2024-06-16 10:58:11,649 mod.rs:752 done loading verification key ✅
INFO ezkl.execute 2024-06-16 10:58:11,685 execute.rs:2373 verify took 0.28
INFO ezkl.execute 2024-06-16 10:58:11,686 execute.rs:2378 verified: true


Proof was successfully verified


We can now create an EVM / `.sol` verifier that can be deployed on chain to verify submitted proofs using a view function.

In [64]:
srs_path="kzg.srs"

abi_path = 'solidity_verifier.abi'
sol_code_path = 'solidity_verifier.sol'

res = await ezkl.create_evm_verifier(
        vk_path,
        settings_path,
        sol_code_path,
        abi_path,
        srs_path
    )
assert res == True

INFO ezkl.pfsys.srs 2024-06-16 10:59:36,222 srs.rs:23 loading srs from "kzg.srs"


INFO ezkl.execute 2024-06-16 10:59:36,266 execute.rs:2467 downsizing params to 17 logrows
INFO ezkl.pfsys 2024-06-16 10:59:36,267 mod.rs:743 loading verification key from "key.vk"
INFO ezkl.pfsys 2024-06-16 10:59:36,286 mod.rs:752 done loading verification key ✅


## Verify on the evm

In [2]:
# Make sure anvil is running locally first
# run with $ anvil -p 3030

# we use the default anvil node here
import json
import ezkl 
import os

address_path = os.path.join("address.json")
sol_code_path = 'solidity_verifier.sol'


res = await ezkl.deploy_evm(
    address_path,
    sol_code_path,
    'http://127.0.0.1:3030'
)

assert res == True

# Save the block hash where contract is deployed
with open(address_path, 'r') as file:
    addr = file.read().rstrip()

In [67]:
# make sure anvil is running locally
# $ anvil -p 3030
proof_path = os.path.join('test.pf')

res = await ezkl.verify_evm(
    addr,
    proof_path,
    "http://127.0.0.1:3030"
)
assert res == True

INFO ezkl.eth 2024-06-16 11:01:06,319 eth.rs:283 using chain 31337


INFO ezkl.eth 2024-06-16 11:01:06,939 eth.rs:595 estimated verify gas cost: 715309
INFO ezkl.execute 2024-06-16 11:01:06,941 execute.rs:1602 Solidity verification result: true


## Now test the both the Torch model and ZKML model preformance by deploying it loaclly.

Execute the 'app.py' file to run a Flask application server

then Exeute the 'ZKP_test_api.py' to start the evaluation process of the predictioon, proof generation and verification process 